In [45]:
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
import os, sys
import nrrd

# Load original NRRD and segmentation

In [47]:
path = '/wmlce/data/medical-datasets/MDT-LIDC-IDRI/NRRDs/LIDC-IDRI-0489/LIDC-IDRI-0489_CT.nrrd'
itkimg = sitk.ReadImage(path)
original_spacing = np.array(itkimg.GetSpacing())
original_img_arr = sitk.GetArrayFromImage(itkimg)
pp_path = '/wmlce/data/medical-datasets/MDT-PP/LIDC-IDRI-0489_img.npy'
pp_img = np.load(pp_path)
pp_spacing = [0.7, 0.7, 1.25]
seg_path = 'seg_0.npy'
seg = np.load(seg_path)
seg_mask = np.zeros(seg.shape).astype(np.uint8)
seg_tresh = 0.5
seg_mask[seg > seg_tresh] = 1

print(f'Original img:\n -> shape: {original_img_arr.shape}\n -> Spacing: {original_spacing}')
print(f'PP img:\n -> shape: {pp_img.shape}\n -> Spacing: {pp_spacing}')
print(f'Seg:\n -> shape: {seg.shape}\n -> Min: {np.min(seg)}\n -> Max: {np.max(seg)}\n -> Mean: {np.mean(seg)}\n -> Std: {np.std(seg)}')
print(f'Seg mask:\n -> shape: {seg_mask.shape}\n -> Unique: {np.unique(seg_mask)}')

Original img:
 -> shape: (141, 512, 512)
 -> Spacing: [0.703125 0.703125 2.5     ]
PP img:
 -> shape: (282, 514, 514)
 -> Spacing: [0.7, 0.7, 1.25]
Seg:
 -> shape: (514, 514, 282)
 -> Min: 0.0
 -> Max: 1.0
 -> Mean: 0.00013387203216552734
 -> Std: 0.0111846923828125
Seg mask:
 -> shape: (514, 514, 282)
 -> Unique: [0 1]


## Reswap and resample seg to original size

In [48]:
from preprocessing import resample_array, resample_array_to_shape
# Reswap segmentation to match original image
seg_mask = np.swapaxes(seg_mask, 0, 2)
seg_mask = np.swapaxes(seg_mask, 1, 2)
# Resample segmentation to original shape
seg_mask, seg_spacing = resample_array_to_shape(seg_mask, pp_spacing, target_shape=original_img_arr.shape)
seg_mask[seg_mask<=0.5] = 0
seg_mask[seg_mask>0] = 1
seg_mask = seg_mask.astype(np.uint8)
print(f'Resampled seg:\n -> shape: {seg_mask.shape}\n -> Spacing: {seg_spacing}\n -> Unique: {np.unique(seg_mask)}')
seg_itk = sitk.GetImageFromArray(seg_mask)
seg_itk.SetOrigin(itkimg.GetOrigin())
seg_itk.SetSpacing(original_spacing)
sitk.WriteImage(seg_itk, 'test.nrrd')

Resampled seg:
 -> shape: (141, 512, 512)
 -> Spacing: [0.7027343749999999, 0.7027343749999999, 2.5]
 -> Unique: [0 1]
